# Ising-Lenz Ergodicity: Rate of convergence to approach to ergodicity temperature dependence

    
     (c) 2013, 2014, 2015, 2016, 2025 Süzen
     GPL v3 

This notebook generates Ising-Lenz 1D, rate of convergence to ergodicity data across temperatures. 
The dataset is called `ising1DrateErgodicity.rds`. 

$\Omega_{G}(t)$ : Modified TM G-metric, ergodic convergence.  
$\Gamma_{G}(t)$ : Rate of ergodic convergence.  
$\Kappa_{G}(t)$ : Inverse rate of ergodic convergence.   

Here we compute $\Gamma$ in a modified fashion, original portion is that ensemble average is 
an exact expression. 

* Workhorse of the data generation is method `performIS_repeat`. The RDS file is simply 
an object of list, each member is the output of this method, a named list. See return of this method.
Average magnetisations are reported there, 40 repeats
* Parameters are hardcoded in paralell method below.
     * Sizes at Metropolis (`transP=1`)  and Glauber (`transP=2`) Dynamics `N=512, 1024, 1536`.
     * Temperature ranges `seq(from = 0.5, by = 0.06, length.out = 16)`.
     * `J=1.0 H=1.0`
     * MC steps `5e6`.


In [ ]:
rm(list=ls()) 
require("isingLenzMC"); # Developed and tested on v0.2.5 and R v4.5.1
library(parallel)       # Core package

source("../src/power_utilities.R");

In [ ]:
parallel_gamma <- function(size, dynamics = 1, num_cores = 8) {
    # Parallel runs
    ntemp <- 16
    max_mc_time <- 5e6
    temperatures <- seq(from = 0.5, by = 0.06, length.out = ntemp)
    mc_gamma <- function(beta) {
        performIS_repeat(beta, J = 1.0, H = 1.0, size, max_mc_time, dynamics, nrepeat = 40)
    }

    current_temp <- 1
    results <- vector("list", ntemp)
    cycle <- as.integer(ntemp / num_cores)
    temp_count <- 0

    for (i in 1:cycle) {
        fromTemp <- temp_count + 1
        toTemp <- temp_count + num_cores
        par_result <- mclapply(temperatures[fromTemp:toTemp], mc_gamma, mc.cores = num_cores)
        for (j in 1:num_cores) {
            results[[current_temp]] <- par_result[[j]]
            current_temp <- current_temp + 1
        }
        temp_count <- temp_count + num_cores
    }
    results
}


In [ ]:
# ~200 min on 8 core MacBook-M2
result <- parallel_gamma(512, dynamics = 1)
result <- c(result, parallel_gamma(512, dynamics = 2))
result <- c(result, parallel_gamma(1024, dynamics = 1))
result <- c(result, parallel_gamma(1024, dynamics = 2))
result <- c(result, parallel_gamma(1536, dynamics = 1))
result <- c(result, parallel_gamma(1536, dynamics = 2))

In [ ]:
saveRDS(result, "../datasets/ising1DrateErgodicity.rds")